In [1]:
import os
import time
import boto3
import json


# Initialize AWS clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=region)


account_id_suffix = account_id[:3]
region_suffix = ''.join([word[0] for word in region.split('-')])
suffix = f"{region_suffix}-{account_id_suffix}"

# Define Knowledge Base parameters
knowledge_base_name_standard = 'fixed-size-chunk-kb'
knowledge_base_name_hierarchical = 'hierarchical-chunk-kb'
knowledge_base_description = "Knowledge Base containing sample synthetic Octank financial data"

print(f"AWS Region: {region}", f"Account ID: {account_id}", f"Suffix: {suffix}", sep='\n')

AWS Region: us-east-1
Account ID: 533267284022
Suffix: ue1-533


In [2]:
from utils.knowledgebase import BedrockKnowledgeBases
from utils.agents import BedrockAgents
from utils.agents import AWSResourceManager
from utils.chat import BedrockChat, SyntheticDataGenerator
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

    
kb = BedrockKnowledgeBases()
agents = BedrockAgents()
chat = BedrockChat()
data = SyntheticDataGenerator()

In [3]:
agent_name=f'rag-agent-{suffix}'
kb_id = "RZJXH5WHWU"

print(f"Agent name: {agent_name}", f"KB id: {kb_id}", sep='\n')


Agent name: rag-agent-ue1-533
KB id: RZJXH5WHWU


## Read eval data from S3

In [4]:
qa_records = data.read_jsonl_from_s3(
    bucket="synthetic-qa-dataset-amzn-10k",
    key="evaluation_data/rag_dataset_prompt_only.jsonl",
    verbose=True
)

2025-01-08 17:36:30,676 [INFO] Reading JSONL from s3://synthetic-qa-dataset-amzn-10k/evaluation_data/rag_dataset_prompt_only.jsonl
2025-01-08 17:36:30,979 [INFO] Successfully read 18 records


In [5]:
qa_records

[{'conversationTurns': [{'prompt': {'content': [{'text': "What is Amazon's principal executive office address?"}]}}]},
 {'conversationTurns': [{'prompt': {'content': [{'text': 'What is the aggregate market value of voting stock held by non-affiliates as of June 30, 2022?'}]}}]},
 {'conversationTurns': [{'prompt': {'content': [{'text': 'What is the fiscal year ended for Amazon.com Inc.?'}]}}]},
 {'conversationTurns': [{'prompt': {'content': [{'text': "What is Amazon's primary focus?"}]}}]},
 {'conversationTurns': [{'prompt': {'content': [{'text': 'What services does AWS offer?'}]}}]},
 {'conversationTurns': [{'prompt': {'content': [{'text': "What are the principal competitive factors in Amazon's retail businesses?"}]}}]},
 {'conversationTurns': [{'prompt': {'content': [{'text': "What is Amazon's total employee count as of December 31, 2022?"}]}}]},
 {'conversationTurns': [{'prompt': {'content': [{'text': 'How many Amazon employees participated in Career Choice?'}]}}]},
 {'conversationTu

## Generate responses fro KB and Agents

In [11]:
response = kb.retrieve_and_generate_from_kb(
    query="What is Amazon's primary focus?",
    kb_id=kb_id,
    num_results=5,
    verbose=False
)

In [7]:
response.generated_text

"Amazon's primary focus is to be Earth's most customer-centric company. They are guided by four principles: customer obsession rather than competitor focus, passion for invention, commitment to operational excellence, and long-term thinking."

In [10]:
print(f"Answer: {response.generated_text}")
print("\nCitations:")
for citation in response.citations:
    print(f"- Source: {citation.source}")
    print(f"  Page: {citation.page_number}")
    print(f"  Text: {citation.text[:200]}...")

Answer: Amazon's primary focus is to be Earth's most customer-centric company. They are guided by four principles: customer obsession rather than competitor focus, passion for invention, commitment to operational excellence, and long-term thinking.

Citations:
- Source: s3://amazon-10k-kb-ue1-533-bucket/AMAZON_2022_10K.pdf
  Page: 2.0
  Text: Certain Relationships and Related Transactions, and Director Independence 71 Item 14. Principal Accountant Fees and Services 71     PART IV Item 15. Exhibits, Financial Statement Schedules 72 Item 16....


In [6]:
user_query = "What is Amazon's primary focus?"

result = agents.invoke(
    agent_name=agent_name, 
    input_text=user_query, 
    verbose=True, 
    trace_level='core',
)


2025-01-08 17:36:52,284 [INFO] Invoking agent 'rag-agent-ue1-533' with input: What is Amazon's primary focus?
---- Step 1 ----
Took 3.6s, using 1777 tokens (in: 1670, out: 107) to complete action.
To answer this question about Amazon's primary focus, I will first search the provided knowledge base for relevant information.
---- Step 2 ----
Took 13.5s, using 2326 tokens (in: 2120, out: 206) to complete action.
Final response:


Amazon's primary focus is on e-commerce and providing a wide range of products and services to customers through its online and physical stores. The company's main source of revenue comes from the sale of merchandise and content purchased for resale, as well as products offered by third-party sellers on its platform.






In addition to serving consumers, Amazon also focuses on providing services and programs for sellers to help them grow their businesses on Amazon's platform. The company also offers a broad set of technology services through its cloud computing

In [7]:
result['citations']

[Citation(text='Certain Relationships and Related Transactions, and Director Independence 71 Item 14. Principal Accountant Fees and Services 71     PART IV Item 15. Exhibits, Financial Statement Schedules 72 Item 16. Form 10-K Summary 74 Signatures 75     2Table of Contents     AMAZON.COM, INC.     PART I     Item 1. Business     This Annual Report on Form 10-K and the documents incorporated herein by reference contain forward-looking statements based on expectations, estimates, and projections as of the date of this filing. Actual results and outcomes may differ materially from those expressed in forward-looking statements. See Item 1A of Part I — “Risk Factors.” As used herein, “Amazon.com,” “we,” “our,” and similar terms include Amazon.com, Inc. and its subsidiaries, unless the context indicates otherwise.     General     We seek to be Earth’s most customer-centric company. We are guided by four principles: customer obsession rather than competitor focus, passion for invention, comm

In [13]:
result['raw_trace']

[{'agentAliasId': 'TSTALIASID',
  'agentId': 'KPJOGYOMCS',
  'agentVersion': 'DRAFT',
  'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:533267284022:agent-alias/KPJOGYOMCS/TSTALIASID'}],
  'sessionId': 'cfcf69cc-3a7a-4193-951b-9b85fbd30bdc',
  'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048,
      'stopSequences': ['</function_calls>', '</answer>', '</error>'],
      'temperature': 0.0,
      'topK': 250,
      'topP': 1.0},
     'text': '{"system":"You are a General AI Assistant helping users with diverse tasks and questions across multiple domains. Your role is to leverage both your general expertise and context-based knowledge retrieved in real-time to provide accurate, relevant, and up-to-date information. You are capable of answering questions with clear explanations on a broad range of topics, including but not limited to technology, science, health, finance, and general knowledge.Response style:1. Maintain a

In [7]:
user_query = "What is the total cost of Octank Financial's property and equipment as of December 31, 2022?"

result = agents.invoke(
    agent_name=agent_name, 
    input_text=user_query, 
    verbose=True, 
    trace_level='core',
)


2025-01-08 15:47:13,412 [INFO] Invoking agent 'rag-agent-ue1-533' with input: What is the total cost of Octank Financial's property and equipment as of December 31, 2022?
---- Step 1 ----
Took 6.0s, using 1841 tokens (in: 1687, out: 154) to complete action.
To find the total cost of Octank Financial's property and equipment as of December 31, 2022, I will need to search the knowledge base for relevant financial information and reports. I will formulate a search query to retrieve this data.
---- Step 2 ----
Took 10.1s, using 2165 tokens (in: 2014, out: 151) to complete action.
The search did not return any relevant information about Octank Financial's property and equipment costs, as the knowledge base appears to only contain data on Amazon. Since I do not have access to financial reports for the company Octank Financial, I cannot provide the requested information.
Final response:
I apologize, but I could not find the total cost of Octank Financial's property and equipment as of Decembe

In [8]:
result.keys()

dict_keys(['response', 'raw_trace', 'trace'])

In [9]:
result['raw_trace']

[{'agentAliasId': 'TSTALIASID',
  'agentId': 'KPJOGYOMCS',
  'agentVersion': 'DRAFT',
  'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:533267284022:agent-alias/KPJOGYOMCS/TSTALIASID'}],
  'sessionId': 'f6ee6872-89f6-469b-a91d-d7cf5e574d6b',
  'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048,
      'stopSequences': ['</function_calls>', '</answer>', '</error>'],
      'temperature': 0.0,
      'topK': 250,
      'topP': 1.0},
     'text': '{"system":"You are a General AI Assistant helping users with diverse tasks and questions across multiple domains. Your role is to leverage both your general expertise and context-based knowledge retrieved in real-time to provide accurate, relevant, and up-to-date information. You are capable of answering questions with clear explanations on a broad range of topics, including but not limited to technology, science, health, finance, and general knowledge.Response style:1. Maintain a

In [7]:
result['raw_trace']

[{'agentAliasId': 'TSTALIASID',
  'agentId': 'XNSPVCAHBV',
  'agentVersion': 'DRAFT',
  'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:533267284022:agent-alias/XNSPVCAHBV/TSTALIASID'}],
  'sessionId': '562ea3f7-0945-456a-8760-4b534e460061',
  'trace': {'orchestrationTrace': {'invocationInput': {'invocationType': 'KNOWLEDGE_BASE',
     'knowledgeBaseLookupInput': {'knowledgeBaseId': '892DWWRLXW',
      'text': "What is the total cost of Octank Financial's property and equipment as of December 31, 2022?"},
     'traceId': '0a337e61-dd98-4149-a031-913c2ed9c27a-0'}}}},
 {'agentAliasId': 'TSTALIASID',
  'agentId': 'XNSPVCAHBV',
  'agentVersion': 'DRAFT',
  'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:533267284022:agent-alias/XNSPVCAHBV/TSTALIASID'}],
  'sessionId': '562ea3f7-0945-456a-8760-4b534e460061',
  'trace': {'orchestrationTrace': {'observation': {'knowledgeBaseLookupOutput': {'retrievedReferences': [{'content': {'text': "The portfolio is classified into

In [12]:
result['enhanced_trace']

[]